In [36]:
import pandas as pd
import numpy as np

**``1``**

In [37]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [38]:
df.isnull().sum()

From_To         0
FlightNumber    2
RecentDelays    0
Airline         0
dtype: int64

In [39]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

**``2``**

**let's fix this missing ``values``**
- **Fixing ``missing`` values with the ``mean`` of ``FlighNumber``**

In [40]:
df.fillna(value=df['FlightNumber'].mean(),inplace=True)

In [41]:
df.isnull().sum()

From_To         0
FlightNumber    0
RecentDelays    0
Airline         0
dtype: int64

**Now, converting ``float`` dtype of ``FlightNumber`` to ``int32`` dtype**
- **``astype('int')``**

In [42]:
df['Flight_Numbers']=df['FlightNumber'].astype(int)

In [43]:
df.drop(columns=['FlightNumber'],inplace=True)

In [44]:
df.dtypes

From_To           object
RecentDelays      object
Airline           object
Flight_Numbers     int32
dtype: object

**``3``**

**``From_To`` column would be ``better`` as  two ``separate`` ``columns``!** 
- **Split each string on the underscore delimiter ``_``**
- **to give a new temporary DataFrame with the correct values**. 

In [45]:
df[['From','To']]=df.From_To.str.split(pat='_', n=-1, expand=True)

In [46]:
df

,From_To,RecentDelays,Airline,Flight_Numbers,From,To
0,LoNDon_paris,"[23, 47]",KLM(!),10045,LoNDon,paris
1,MAdrid_miLAN,[],<Air France> (12),10065,MAdrid,miLAN
2,londON_StockhOlm,"[24, 43, 87]",(British Airways. ),10065,londON,StockhOlm
3,Budapest_PaRis,[13],12. Air France,10065,Budapest,PaRis
4,Brussels_londOn,"[67, 32]","""Swiss Air""",10085,Brussels,londOn


**``4``**

- **Notice how the capitalisation of the city names is all mixed up in this** 
- **temporary DataFrame. Standardise the strings**
- **so that only the first letter is** 
- **uppercase (e.g. "londON" should become "London".)**

In [47]:
df['From']=df['From'].str.capitalize()
df['To']=df['To'].str.capitalize()

**``5``**

**``Delete`` the ``From_To`` column from df**

In [48]:
df.drop(columns=['From_To'],inplace=True)

In [49]:
df

,RecentDelays,Airline,Flight_Numbers,From,To
0,"[23, 47]",KLM(!),10045,London,Paris
1,[],<Air France> (12),10065,Madrid,Milan
2,"[24, 43, 87]",(British Airways. ),10065,London,Stockholm
3,[13],12. Air France,10065,Budapest,Paris
4,"[67, 32]","""Swiss Air""",10085,Brussels,London


**``6``**

- **In the ``RecentDelays`` column,**
- **the values have been entered into the DataFrame as a list.**
- **We would like ``each`` ``first`` value in its own column,**
- **each second value in its own column, and so on.**
- **If there ``isn't`` an ``Nth`` value, the value should be ``NaN``.**


- **Expand the Series of lists into a DataFrame named delays, rename the columns**

- **delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.**

In [70]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [71]:
rows=[]
_=df.apply(lambda row:[rows.append([row['Airline'],row['FlightNumber'],nn,row['From_To']])
                      for nn in row.RecentDelays],axis=1)

In [72]:
rows

[['KLM(!)', 10045.0, 23, 'LoNDon_paris'],
 ['KLM(!)', 10045.0, 47, 'LoNDon_paris'],
 ['(British Airways. )', 10065.0, 24, 'londON_StockhOlm'],
 ['(British Airways. )', 10065.0, 43, 'londON_StockhOlm'],
 ['(British Airways. )', 10065.0, 87, 'londON_StockhOlm'],
 ['12. Air France', nan, 13, 'Budapest_PaRis'],
 ['"Swiss Air"', 10085.0, 67, 'Brussels_londOn'],
 ['"Swiss Air"', 10085.0, 32, 'Brussels_londOn']]

In [73]:
df_new=pd.DataFrame(rows,columns=df.columns)
print("New : ",df_new)
print("-"*90)
print("Old : ",df)

New :                 From_To  FlightNumber  RecentDelays           Airline
0               KLM(!)       10045.0            23      LoNDon_paris
1               KLM(!)       10045.0            47      LoNDon_paris
2  (British Airways. )       10065.0            24  londON_StockhOlm
3  (British Airways. )       10065.0            43  londON_StockhOlm
4  (British Airways. )       10065.0            87  londON_StockhOlm
5       12. Air France           NaN            13    Budapest_PaRis
6          "Swiss Air"       10085.0            67   Brussels_londOn
7          "Swiss Air"       10085.0            32   Brussels_londOn
------------------------------------------------------------------------------------------
Old :              From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. 

In [74]:
delay_df=pd.DataFrame(df['RecentDelays'].values.tolist())
delay_df

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [75]:
length_cols=delay_df.shape[1]
length_cols

3

In [77]:
col_list=[]
col_dict={}
for i in range(length_cols):
    key = delay_df.columns[i]
    Value="Delay"+str(i+1)
    col_dict[key]=Value
col_dict    

{0: 'Delay1', 1: 'Delay2', 2: 'Delay3'}

In [78]:
delay_df.columns[2]

2

In [79]:
delay_df.columns[2]

2

# **Finished**

**``Explanation`` of concept used in ``for`` loop**

In [66]:
color={"name":'Amit','age':19}

In [80]:
c={}
c['dob']='1-1-2004'
c

{'dob': '1-1-2004'}